<a href="https://colab.research.google.com/github/shichaog/transformer/blob/main/Finetuning_LLama_2_0_on_Colab_with_1_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install huggingface_hub
!pip install transformers==4.31.0
!pip install accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 trl==0.4.7
!pip install sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

colab上每次登录，GPU上有一些基础的python库，但是有些没有，需要安装，比如Huggingface_hub。

In [4]:
from huggingface_hub import login

login()


如果使用私有数据，需要登录Huggingface账号，开源的数据集也可以不登录。

In [5]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Activate 4-bit precision base model loading
use_4bit = True
# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False


bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

model_name = "meta-llama/Llama-2-7b-chat-hf"
#Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# needed for llama tokenizer
tokenizer.pad_token = tokenizer.eos_token

####Below is for mlabonne/guanaco-llama2-1k dataset
#tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training


#Load the entire model on the GPU 0
device_map = {"": 0}

#Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

导入预训练模型. 使用transformers库的AutoTokenizer类和 AutoModelForCausalLM 类自动下载和创建模型实例. The BitsAndBytesConfig类用于模型的量化参数设置，比如4-bit是量化位数，torch.bfloat16是微调时用的数据类型。

In [6]:
from peft import prepare_model_for_kbit_training
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

从peft库导入prepare_model_for_kbit_training函数，并使用该函数进行k-bit量化前准备. gradient_checkpointing_enable() 函数是能了在训练阶段可以降低内存使用的梯度 checkpointing特性。

In [7]:
def print_trainable_parameters(model):
  """

  Prints the number of trainable parameters in the model.

  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
      print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
      )

from peft import LoraConfig, get_peft_model

# LoRA attention dimension 64， 8
lora_r = 8

# Alpha parameter for LoRA scaling 16,32
lora_alpha = 32

# Dropout probability for LoRA layers 0.1 0.05
lora_dropout = 0.1

peft_config = LoraConfig(
  r=lora_r,
  lora_alpha=lora_alpha,
  target_modules=["q_proj","v_proj"],
  lora_dropout=lora_dropout,
  bias="none",
  task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)
print_trainable_parameters(model)



trainable params: 32768 || all params: 139493376 || trainable%: 0.02349072116513977
trainable params: 65536 || all params: 139526144 || trainable%: 0.04697040864255519
trainable params: 98304 || all params: 156336128 || trainable%: 0.06287989939216097
trainable params: 131072 || all params: 156368896 || trainable%: 0.08382229673093043
trainable params: 163840 || all params: 240820224 || trainable%: 0.06803415314487873
trainable params: 196608 || all params: 240852992 || trainable%: 0.08162987653481174
trainable params: 229376 || all params: 257662976 || trainable%: 0.08902171493975138
trainable params: 262144 || all params: 257695744 || trainable%: 0.10172616587722923
trainable params: 294912 || all params: 342147072 || trainable%: 0.086194512282718
trainable params: 327680 || all params: 342179840 || trainable%: 0.09576250897773522
trainable params: 360448 || all params: 358989824 || trainable%: 0.10040618867235634
trainable params: 393216 || all params: 359022592 || trainable%: 0.109

`print_trainable_parameters`函数用于打印模型可训练参数. 从peft库导入 `LoraConfig` 和 `get_peft_model`函数。`LoraConfig`用于配置缩减训练参数的LORA (Low Rank Approximation)方法。`get_peft_model`将LORA方法应用于模型. 打印的是模型可训练参数的情况。

从终端输出可以看到使用LORA方法后约11%的参数才会被微调时更新， 这大大降低了内存，不同的LORA参数会需要不同的内存，下图中的两种配置，分别对应了训练的时候需要内存情况。

In [8]:
from datasets import load_dataset
dataset = load_dataset("Abirate/english_quotes")
dataset = dataset.map(lambda samples: tokenizer(samples["quote"]), batched=True)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

从Huggingface的`datasets`库导入`load_dataset`函数， 用其加载`"Abirate/english_quotes"`数据集中的`“quotes”`字段，然后使用LLaMA tokenizer对其tokenize化。

In [13]:
import transformers

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use, paged_adamw_8bit paged_adamw_32bit etc...
optim = "paged_adamw_8bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

# Fine-tuned model name
new_model = "llama-2-7b-shichaog"

# Set training parameters
training_arguments = transformers.TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

## needed for llama tokenizer
tokenizer.pad_token = tokenizer.eos_token
trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset["train"],
    # args=transformers.TrainingArguments(
    #     per_device_train_batch_size=1,
    #     gradient_accumulation_steps=4,
    #     warmup_steps=2,
    #     max_steps=10,
    #     learning_rate=2e-4,
    #     fp16=True,
    #     logging_steps=1,
    #     output_dir="outputs",
    #     optim="paged_adamw_8bit"
    #     ),
    args=training_arguments,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False # silence the warnings. Please re-enable for inference!
trainer.train()
trainer.model.save_pretrained(new_model)

Step,Training Loss
25,1.098700
50,0.917400
75,0.998600
100,0.923200


KeyboardInterrupt: ignored

这里使用了transformers库中的Trainer类，使用模型, 训练数据集, 以及训练参数对Trainer实例化，训练数据集设置了训练时的各种参数，比如 batch size, learning rate, and 优化算法 (paged_adamw_8bit)。  `DataCollatorForLanguageModeling` 用于整理和批处理（batch）标记化数据。 最终调用trainer.train()方法开启微调训练。在后文又给了基于trl库的更简单的接口。

In [12]:
################################################################################
# SFT parameters
################################################################################
from trl import SFTTrainer
# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

# Set supervised fine-tuning parameters from trl library
trainer2 = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    peft_config=peft_config,
    dataset_text_field="quote",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer2.train()

# Save trained model
trainer2.model.save_pretrained(new_model)

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

Step,Training Loss
25,1.351300
50,1.389100
75,1.366300
100,1.425100


KeyboardInterrupt: ignored

这段代码和上一段使用transformers库的Trainer是一样的意义和作用，这里的SFTTrainer是对上面Trainer的封装，参数的意义都是一样的。因为trl库支持了PPO之类的RLHF，所以把SFT也支持了会使trl库更完备一些。